![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/4bd64c1deb3c4ffb8ca7ff93c1f1497d).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import LabelEncoder

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTrain_carListings.zip')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTest_carListings.zip', index_col=0)

In [4]:
# Visualización datos de entrenamiento
dataTraining.head()

,Price,Year,Mileage,State,Make,Model
0,34995,2017,9913,FL,Jeep,Wrangler
1,37895,2015,20578,OH,Chevrolet,Tahoe4WD
2,18430,2012,83716,TX,BMW,X5AWD
3,24681,2014,28729,OH,Cadillac,SRXLuxury
4,26998,2013,64032,CO,Jeep,Wrangler


In [5]:
# Visualización datos de test
dataTesting.head()

,Year,Mileage,State,Make,Model
ID,,,,,
0,2014,31909,MD,Nissan,MuranoAWD
1,2017,5362,FL,Jeep,Wrangler
2,2014,50300,OH,Ford,FlexLimited
3,2004,132160,WA,BMW,5
4,2015,25226,MA,Jeep,Grand


# FUNCIONES

In [6]:
def guardar(ypred,nombre):
    ypred = pd.DataFrame(ypred,index=dataTesting.index, columns=['Price'])
    # Guardar predicciones en formato exigido en la competencia de kaggle
    archivo = nombre + '.csv'
    ypred.to_csv(archivo, index_label='ID')
    ypred.head()

# Análisis exploratorio

In [7]:
dTr = dataTraining.copy()

In [8]:
dTr.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Price,400000.0,NaN,NaN,NaN,21146.919312,10753.66494,5001.0,13499.0,18450.0,26999.0,79999.0
Year,400000.0,NaN,NaN,NaN,2013.198125,3.292326,1997.0,2012.0,2014.0,2016.0,2018.0
Mileage,400000.0,NaN,NaN,NaN,55072.956895,40881.018845,5.0,25841.0,42955.0,77433.0,2457832.0
State,400000,51,TX,45918,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Make,400000,38,Ford,62899,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Model,400000,525,Silverado,18085,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
np.sort(dTr.State.unique())

array([' AK', ' AL', ' AR', ' AZ', ' CA', ' CO', ' CT', ' DC', ' DE',
       ' FL', ' GA', ' HI', ' IA', ' ID', ' IL', ' IN', ' KS', ' KY',
       ' LA', ' MA', ' MD', ' ME', ' MI', ' MN', ' MO', ' MS', ' MT',
       ' NC', ' ND', ' NE', ' NH', ' NJ', ' NM', ' NV', ' NY', ' OH',
       ' OK', ' OR', ' PA', ' RI', ' SC', ' SD', ' TN', ' TX', ' UT',
       ' VA', ' VT', ' WA', ' WI', ' WV', ' WY'], dtype=object)

In [10]:
np.sort(dTr.Make.unique())

array(['Acura', 'Audi', 'BMW', 'Bentley', 'Buick', 'Cadillac',
       'Chevrolet', 'Chrysler', 'Dodge', 'FIAT', 'Ford', 'Freightliner',
       'GMC', 'Honda', 'Hyundai', 'INFINITI', 'Jaguar', 'Jeep', 'Kia',
       'Land', 'Lexus', 'Lincoln', 'MINI', 'Mazda', 'Mercedes-Benz',
       'Mercury', 'Mitsubishi', 'Nissan', 'Pontiac', 'Porsche', 'Ram',
       'Scion', 'Subaru', 'Suzuki', 'Tesla', 'Toyota', 'Volkswagen',
       'Volvo'], dtype=object)

In [11]:
np.sort(dTr.Model.unique())

array(['1', '15002WD', '15004WD', '1500Laramie', '1500Tradesman', '200LX',
       '200Limited', '200S', '200Touring', '25002WD', '25004WD', '3',
       '300300C', '300300S', '3004dr', '300Base', '300Limited',
       '300Touring', '35004WD', '350Z2dr', '4Runner2WD', '4Runner4WD',
       '4Runner4dr', '4RunnerLimited', '4RunnerRWD', '4RunnerSR5',
       '4RunnerTrail', '5', '500Pop', '6', '7', '911', '9112dr', 'A34dr',
       'A44dr', 'A64dr', 'A8', 'AcadiaAWD', 'AcadiaFWD', 'Accent4dr',
       'Accord', 'AccordEX', 'AccordEX-L', 'AccordLX', 'AccordLX-S',
       'AccordSE', 'Altima4dr', 'Armada2WD', 'Armada4WD', 'Avalanche2WD',
       'Avalanche4WD', 'Avalon4dr', 'AvalonLimited', 'AvalonTouring',
       'AvalonXLE', 'Azera4dr', 'Boxster2dr', 'C-Class4dr', 'C-ClassC',
       'C-ClassC300', 'C-ClassC350', 'C702dr', 'CC4dr', 'CR-V2WD',
       'CR-V4WD', 'CR-VEX', 'CR-VEX-L', 'CR-VLX', 'CR-VSE', 'CR-ZEX',
       'CT', 'CTCT', 'CTS', 'CTS-V', 'CTS4dr', 'CX-7FWD', 'CX-9AWD',
       'CX-9FWD', 

In [12]:
dTr.isnull().sum() / dTr.shape[0] * 100

Price      0.0
Year       0.0
Mileage    0.0
State      0.0
Make       0.0
Model      0.0
dtype: float64

In [13]:
dTr.groupby(["Model"],sort=False).count()

,Price,Year,Mileage,State,Make
Model,,,,,
Wrangler,5914,5914,5914,5914,5914
Tahoe4WD,1912,1912,1912,1912,1912
X5AWD,601,601,601,601,601
SRXLuxury,1270,1270,1270,1270,1270
3,5516,5516,5516,5516,5516
...,...,...,...,...,...
PriusOne,75,75,75,75,75
Element2WD,57,57,57,57,57
"Excursion137""",92,92,92,92,92


In [14]:
dTr.groupby(["Make"],sort=False).count()

,Price,Year,Mileage,State,Model
Make,,,,,
Jeep,24369,24369,24369,24369,24369
Chevrolet,58383,58383,58383,58383,58383
BMW,12326,12326,12326,12326,12326
Cadillac,5414,5414,5414,5414,5414
Mercedes-Benz,7575,7575,7575,7575,7575
Toyota,45941,45941,45941,45941,45941
Buick,4842,4842,4842,4842,4842
Dodge,16159,16159,16159,16159,16159
Volkswagen,11110,11110,11110,11110,11110


In [15]:
dTr.groupby(["State"],sort=False).count()

,Price,Year,Mileage,Make,Model
State,,,,,
FL,33759,33759,33759,33759,33759
OH,11364,11364,11364,11364,11364
TX,45918,45918,45918,45918,45918
CO,10875,10875,10875,10875,10875
ME,965,965,965,965,965
WA,10900,10900,10900,10900,10900
CT,4942,4942,4942,4942,4942
CA,36534,36534,36534,36534,36534
LA,3847,3847,3847,3847,3847


# PREPROCESAMIENTO

In [16]:
leState = LabelEncoder()
leState.fit(dTr["State"])
dTr["State"]=leState.transform(dTr.State)

In [17]:
leMake = LabelEncoder()
leMake.fit(dTr["Make"])
dTr["Make"]=leMake.transform(dTr.Make)

In [18]:
leModel = LabelEncoder()
leModel.fit(dTr["Model"])
dTr["Model"]=leModel.transform(dTr.Model)

In [19]:
dTe = dataTesting.copy()
dTe

,Year,Mileage,State,Make,Model
ID,,,,,
0,2014,31909,MD,Nissan,MuranoAWD
1,2017,5362,FL,Jeep,Wrangler
2,2014,50300,OH,Ford,FlexLimited
3,2004,132160,WA,BMW,5
4,2015,25226,MA,Jeep,Grand
...,...,...,...,...,...
99995,2015,82719,TX,GMC,Savana
99996,2015,19711,UT,BMW,5
99997,2016,48049,CA,Nissan,MuranoAWD


In [20]:
dTe["State"]=leState.transform(dTe.State)
dTe["Make"]=leMake.transform(dTe.Make)
dTe["Model"]=leModel.transform(dTe.Model)

In [21]:
XTotal = dTr.loc[:,dTr.columns!='Price']
yTotal = dTr.loc[:,dTr.columns=='Price']

In [22]:
XTrain, XTest, yTrain, yTest = train_test_split(XTotal, yTotal, test_size=0.30, random_state=0)

# CALIBRACIÓN DEL MODELO

## Random Forest

In [120]:
np.random.seed(123)
max_depth_range = range(15,20)
feature_range = range(4, 7)
estimator_range = range(300, 310, 1)

In [121]:
for max_depth in max_depth_range:
    reg = RandomForestRegressor(max_depth = max_depth, max_features = 5, n_estimators = 301, n_jobs=-1)
    scores = cross_val_score(reg, XTrain, yTrain, cv = 5, scoring = 'neg_mean_squared_error', n_jobs=-1)
    rmse_scoresRF1.append(np.mean(np.sqrt(np.abs(scores))))
    parametrosRF1.append((max_depth,feature,estimator))

In [123]:
for feature in feature_range:
    reg = RandomForestRegressor(max_depth = 16, max_features = feature, n_estimators = 250, n_jobs=-1)
    scores = cross_val_score(reg, XTrain, yTrain, cv = 5, scoring = 'neg_mean_squared_error', n_jobs=-1)
    rmse_scoresRF1.append(np.mean(np.sqrt(np.abs(scores))))
    parametrosRF1.append((max_depth,feature,estimator))

In [125]:
for estimator in estimator_range:
    reg = RandomForestRegressor(max_depth = 16, max_features = 5, n_estimators = estimator, n_jobs=-1)
    scores = cross_val_score(reg, XTrain, yTrain, cv = 5, scoring = 'neg_mean_squared_error', n_jobs=-1)
    rmse_scoresRF1.append(np.mean(np.sqrt(np.abs(scores))))
    parametrosRF1.append((max_depth,feature,estimator))

In [126]:
ind_menorRMSE = rmse_scoresRF1.index(min(rmse_scoresRF1))
minParametros = parametrosRF1[ind_menorRMSE]
print('Los parametros optimos del modelo son: ')
print('max_depth= ' + str(parametrosRF1[ind_menorRMSE][0]))
print('max_features= ' + str(parametrosRF1[ind_menorRMSE][1]))
print('n_estimators= ' + str(parametrosRF1[ind_menorRMSE][2]))

Los parametros optimos del modelo son: 
max_depth= 16
max_features= 5
n_estimators= 301


In [36]:
np.random.seed(123)

# Implementación y ajuste del modelo de regresión de Random Forest calibrado con los datos de entrenamiento
regRF1 = RandomForestRegressor(max_depth = 16, max_features = 5, n_estimators = 301, n_jobs=-1).fit(XTrain, yTrain)

ypredRF1 = regRF1.predict(XTest)

RMSE_RF1 = round(mse(ypredRF1, yTest, squared = False))
MSE_RF1 = round(mse(ypredRF1, yTest))
MAE_RF1 = round(mae(ypredRF1, yTest))

# Cálculo del desempeño del modelo utilizando el RMSE y el MAE
print(f'El RMSE del modelo es de {RMSE_RF1}')
print(f'El MSE del modelo es de {MSE_RF1}')
print(f'El MAE del modelo es de {MAE_RF1}')

El RMSE del modelo es de 3630
El MSE del modelo es de 13174637
El MAE del modelo es de 2306


In [37]:
regRF11 = RandomForestRegressor(max_depth = 16, max_features = 5, n_estimators = 301, n_jobs=-1).fit(XTotal, yTotal)

ypredRF11 = regRF11.predict(dTe)

## XGBoost

## Modelo base LR, Gamma, colsample

In [111]:
np.random.seed(123)
lr_range = np.arange(0.4, 0.6, 0.01)
gamma_range = range(385, 400, 1)
colsample_bytree_range = np.arange(0.38, 0.5, 0.01)
#rmse_scoresXGB1=[]
#parametrosXGB1=[]

In [112]:
gamma = 390
colsample_bytree = 0.41
for lr in lr_range:
    reg = XGBRegressor(learning_rate = lr, gamma = gamma, colsample_bytree = colsample_bytree)
    scores = cross_val_score(reg, XTrain, yTrain, cv = 5, scoring = 'neg_mean_squared_error', n_jobs=-1)
    rmse_scoresXGB1.append(np.mean(np.sqrt(np.abs(scores))))
    parametrosXGB1.append((lr,gamma,colsample_bytree))

In [104]:
lr = 0.48
colsample_bytree = 0.41
for gamma in gamma_range:
    reg = XGBRegressor(learning_rate = lr, gamma = gamma, colsample_bytree = colsample_bytree)
    scores = cross_val_score(reg, XTrain, yTrain, cv = 5, scoring = 'neg_mean_squared_error', n_jobs=-1)
    rmse_scoresXGB1.append(np.mean(np.sqrt(np.abs(scores))))
    parametrosXGB1.append((lr,gamma,colsample_bytree))

In [109]:
lr = 0.48
gamma = 390
for colsample_bytree in colsample_bytree_range:
    reg = XGBRegressor(learning_rate = lr, gamma = gamma, colsample_bytree = colsample_bytree)
    scores = cross_val_score(reg, XTrain, yTrain, cv = 5, scoring = 'neg_mean_squared_error', n_jobs=-1)
    rmse_scoresXGB1.append(np.mean(np.sqrt(np.abs(scores))))
    parametrosXGB1.append((lr,gamma,colsample_bytree))

In [113]:
ind_menorRMSE = rmse_scoresXGB1.index(min(rmse_scoresXGB1))
print('Los parametros optimos del modelo son: ')
print('Learning rate= ' + str(parametrosXGB1[ind_menorRMSE][0]))
print('Gamma= ' + str(parametrosXGB1[ind_menorRMSE][1]))
print('Colsample tree= ' + str(parametrosXGB1[ind_menorRMSE][2]))

Los parametros optimos del modelo son: 
Learning rate= 0.5400000000000001
Gamma= 390
Colsample tree= 0.41


In [35]:
np.random.seed(123)

# Implementación y ajuste del modelo de regresión de Random Forest calibrado con los datos de entrenamiento
regXGB1 = XGBRegressor(learning_rate = 0.54, gamma = 390, colsample_bytree = 0.41).fit(XTrain, yTrain)

ypredXGB1 = regXGB1.predict(XTest)

RMSE_XGB1 = round(mse(ypredXGB1, yTest, squared = False))
MSE_XGB1 = round(mse(ypredXGB1, yTest))
MAE_XGB1 = round(mae(ypredXGB1, yTest))

# Cálculo del desempeño del modelo utilizando el RMSE y el MAE
print(f'El RMSE del modelo es de {RMSE_XGB1}')
print(f'El MSE del modelo es de {MSE_XGB1}')
print(f'El MAE del modelo es de {MAE_XGB1}')

El RMSE del modelo es de 3793
El MSE del modelo es de 14384443
El MAE del modelo es de 2485


In [146]:
regXGB11 = XGBRegressor(learning_rate = 0.54, gamma = 390, colsample_bytree = 0.41).fit(XTotal, yTotal)

ypredXGB11 = regXGB11.predict(dTe)

## Modelo mejorado max_depth

In [28]:
np.random.seed(123)
eta_range = np.arange(0.01, 0.7, 0.01)
gamma_range = range(1, 400, 20)
colsample_bytree_range = np.arange(0.05, 0.5, 0.02)
max_depth_range = range(1,20,1)
estimator_range = range(100, 500, 20)
rmse_scoresXGB2=[]
parametrosXGB2=[]

In [30]:
for eta in eta_range:
    reg = XGBRegressor(eta = eta, n_jobs=-1)
    scores = cross_val_score(reg, XTrain, yTrain, cv = 5, scoring = 'neg_mean_squared_error', n_jobs=-1)
    rmse_scoresXGB2.append(np.mean(np.sqrt(np.abs(scores))))
    parametrosXGB2.append((eta,0,1,6))

In [175]:
gamma = 390
colsample_bytree = 0.41
max_depth = 10
n_estimators = 200
for lr in lr_range:
    reg = XGBRegressor(learning_rate = lr, gamma = gamma, colsample_bytree = colsample_bytree, max_depth = max_depth, n_estimators = n_estimators,n_jobs=-1)
    scores = cross_val_score(reg, XTrain, yTrain, cv = 5, scoring = 'neg_mean_squared_error', n_jobs=-1)
    rmse_scoresXGB2.append(np.mean(np.sqrt(np.abs(scores))))
    parametrosXGB2.append((lr,gamma,colsample_bytree,max_depth,n_estimators))

In [178]:
lr = 0.5
colsample_bytree = 0.41
max_depth = 2
n_estimators = 200
for gamma in gamma_range:
    reg = XGBRegressor(learning_rate = lr, gamma = gamma, colsample_bytree = colsample_bytree,max_depth = max_depth, n_estimators = n_estimators,n_jobs=-1)
    scores = cross_val_score(reg, XTrain, yTrain, cv = 5, scoring = 'neg_mean_squared_error', n_jobs=-1)
    rmse_scoresXGB1.append(np.mean(np.sqrt(np.abs(scores))))
    parametrosXGB1.append((lr,gamma,colsample_bytree))

In [179]:
lr = 0.5
gamma = 390
max_depth = 2
n_estimators = 200
for colsample_bytree in colsample_bytree_range:
    reg = XGBRegressor(learning_rate = lr, gamma = gamma, colsample_bytree = colsample_bytree,max_depth = max_depth, n_estimators = n_estimators,n_jobs=-1)
    scores = cross_val_score(reg, XTrain, yTrain, cv = 5, scoring = 'neg_mean_squared_error', n_jobs=-1)
    rmse_scoresXGB1.append(np.mean(np.sqrt(np.abs(scores))))
    parametrosXGB1.append((lr,gamma,colsample_bytree))

In [180]:
lr = 0.5
gamma = 390
colsample_bytree = 0.41
n_estimators = 200
for max_depth in max_depth_range:
    reg = XGBRegressor(learning_rate = lr, gamma = gamma, colsample_bytree = colsample_bytree,max_depth = max_depth, n_estimators = n_estimators,n_jobs=-1)
    scores = cross_val_score(reg, XTrain, yTrain, cv = 5, scoring = 'neg_mean_squared_error', n_jobs=-1)
    rmse_scoresXGB1.append(np.mean(np.sqrt(np.abs(scores))))
    parametrosXGB1.append((lr,gamma,colsample_bytree))

In [181]:
lr = 0.5
gamma = 390
colsample_bytree = 0.41
max_depth = 2
for n_estimators in estimator_range:
    reg = XGBRegressor(learning_rate = lr, gamma = gamma, colsample_bytree = colsample_bytree,max_depth = max_depth, n_estimators = n_estimators,n_jobs=-1)
    scores = cross_val_score(reg, XTrain, yTrain, cv = 5, scoring = 'neg_mean_squared_error', n_jobs=-1)
    rmse_scoresXGB1.append(np.mean(np.sqrt(np.abs(scores))))
    parametrosXGB1.append((lr,gamma,colsample_bytree))

In [32]:
ind_menorRMSE = rmse_scoresXGB2.index(min(rmse_scoresXGB2))
print('Los parametros optimos del modelo son: ')
print('Learning rate= ' + str(parametrosXGB2[ind_menorRMSE][0]))
print('Gamma= ' + str(parametrosXGB2[ind_menorRMSE][1]))
print('Colsample tree= ' + str(parametrosXGB2[ind_menorRMSE][2]))
print('Max depth= ' + str(parametrosXGB2[ind_menorRMSE][3]))
#print('N estimators = ' + str(parametrosXGB2[ind_menorRMSE][4]))

Los parametros optimos del modelo son: 
Learning rate= 0.63
Gamma= 0
Colsample tree= 1
Max depth= 6


In [31]:
np.random.seed(123)

# Implementación y ajuste del modelo de regresión de Random Forest calibrado con los datos de entrenamiento
regXGB2 = XGBRegressor(eta = 0.54, gamma = 390, colsample_bytree = 0.41,max_depth = 8, n_estimators = 301, n_jobs=-1).fit(XTrain, yTrain)
ypredTrXGB2 = regXGB2.predict(XTrain)
ypredXGB2 = regXGB2.predict(XTest)

RMSE_XGB2 = round(mse(ypredXGB2, yTest, squared = False))
MSE_XGB2 = round(mse(ypredXGB2, yTest))
MAE_XGB2 = round(mae(ypredXGB2, yTest))

RMSE_XGB2Tr = round(mse(ypredTrXGB2, yTrain, squared = False))
MSE_XGB2Tr = round(mse(ypredTrXGB2, yTrain))
MAE_XGB2Tr = round(mae(ypredTrXGB2, yTrain))

# Cálculo del desempeño del modelo utilizando el RMSE y el MAE
print(f'El RMSE del modelo es de {RMSE_XGB2}')
print(f'El MSE del modelo es de {MSE_XGB2}')
print(f'El MAE del modelo es de {MAE_XGB2}')

print(f'El RMSE del modelo es de {RMSE_XGB2Tr}')
print(f'El MSE del modelo es de {MSE_XGB2Tr}')
print(f'El MAE del modelo es de {MAE_XGB2Tr}')

El RMSE del modelo es de 3564
El MSE del modelo es de 12702364
El MAE del modelo es de 2269
El RMSE del modelo es de 3127
El MSE del modelo es de 9778326
El MAE del modelo es de 2017


In [39]:
np.random.seed(123)

# Implementación y ajuste del modelo de regresión de Random Forest calibrado con los datos de entrenamiento
regXGB2 = XGBRegressor(learning_rate = 0.1, gamma = 200, colsample_bytree = 0.4,max_depth = 9, n_estimators = 300, n_jobs=-1).fit(XTrain, yTrain)
ypredTrXGB2 = regXGB2.predict(XTrain)
ypredXGB2 = regXGB2.predict(XTest)

RMSE_XGB2 = round(mse(ypredXGB2, yTest, squared = False))
MSE_XGB2 = round(mse(ypredXGB2, yTest))
MAE_XGB2 = round(mae(ypredXGB2, yTest))

RMSE_XGB2Tr = round(mse(ypredTrXGB2, yTrain, squared = False))
MSE_XGB2Tr = round(mse(ypredTrXGB2, yTrain))
MAE_XGB2Tr = round(mae(ypredTrXGB2, yTrain))

# Cálculo del desempeño del modelo utilizando el RMSE y el MAE
print(f'El RMSE del modelo es de {RMSE_XGB2}')
print(f'El MSE del modelo es de {MSE_XGB2}')
print(f'El MAE del modelo es de {MAE_XGB2}')

print(f'El RMSE del modelo es de {RMSE_XGB2Tr}')
print(f'El MSE del modelo es de {MSE_XGB2Tr}')
print(f'El MAE del modelo es de {MAE_XGB2Tr}')

El RMSE del modelo es de 3558
El MSE del modelo es de 12660042
El MAE del modelo es de 2267
El RMSE del modelo es de 3302
El MSE del modelo es de 10905638
El MAE del modelo es de 2130


In [40]:
regXGB21 = XGBRegressor(learning_rate = 0.1, gamma = 200, colsample_bytree = 0.4,max_depth = 9, n_estimators = 300, n_jobs=-1).fit(XTotal, yTotal)

ypredXGB21 = regXGB21.predict(dTe)

# RESULTADOS

In [152]:
ypred_RF_XGB = (ypredRF1 + ypredXGB1)/2

RMSE_RFXGB = round(mse(ypred_RF_XGB, yTest, squared = False))
MSE_RFXGB1 = round(mse(ypred_RF_XGB, yTest))
MAE_RFXGB1 = round(mae(ypred_RF_XGB, yTest))

# Cálculo del desempeño del modelo utilizando el RMSE y el MAE
print(f'El RMSE del modelo es de {RMSE_RFXGB}')
print(f'El MSE del modelo es de {MSE_RFXGB1}')
print(f'El MAE del modelo es de {MAE_RFXGB1}')

El RMSE del modelo es de 3577
El MSE del modelo es de 12792787
El MAE del modelo es de 2284


In [148]:
# Predicción del conjunto de test - acá se genera un número aleatorio como ejemplo
np.random.seed(42)
ypredRF11 = pd.DataFrame(ypredRF11,index=dTe.index, columns=['Price'])

In [150]:
# Guardar predicciones en formato exigido en la competencia de kaggle
ypredRF11.to_csv('RF11.csv', index_label='ID')
ypredRF11.head()

,Price
ID,
0,20968.707204
1,37246.444303
2,24369.319006
3,8910.731448
4,30726.397024


In [151]:
# Predicción del conjunto de test - acá se genera un número aleatorio como ejemplo
np.random.seed(42)
ypredXGB11 = pd.DataFrame(ypredXGB11,index=dTe.index, columns=['Price'])
# Guardar predicciones en formato exigido en la competencia de kaggle
ypredXGB11.to_csv('XGB11.csv', index_label='ID')
ypredXGB11.head()

,Price
ID,
0,20687.025391
1,37395.105469
2,20049.353516
3,6455.743164
4,29685.115234


In [153]:
ypredRF1XGB1 = (ypredRF11 + ypredXGB11)/2
ypredRF1XGB1 = pd.DataFrame(ypredRF1XGB1,index=dTe.index, columns=['Price'])
# Guardar predicciones en formato exigido en la competencia de kaggle
ypredRF1XGB1.to_csv('RFXGB11.csv', index_label='ID')
ypredRF1XGB1.head()

,Price
ID,
0,20827.866297
1,37320.774886
2,22209.336261
3,7683.237306
4,30205.756129


In [41]:
ypredXGB2

array([18138.389, 28519.514, 20517.768, ..., 29416.78 , 11741.732,
       15211.2  ], dtype=float32)

In [42]:
ypredRF1XGB2 = (ypredRF11 + ypredXGB21)/2
ypredRF1XGB2 = pd.DataFrame(ypredRF1XGB2,index=dTe.index, columns=['Price'])
# Guardar predicciones en formato exigido en la competencia de kaggle
ypredRF1XGB2.to_csv('RF1XGB2.csv', index_label='ID')
ypredRF1XGB2.head()

,Price
ID,
0,21108.075784
1,37808.630791
2,23706.953663
3,8945.155668
4,30557.682974
